In [1]:
#from sklearn.model_selection import GridSearchCV
# from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam, SGD, RMSprop, Adagrad
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D, Conv2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras
import tensorflow as tf
import keras_tuner as kt

fer_directory = 'data/FER2013'
ck_directory = 'data/CK+'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Using TensorFlow backend
Num GPUs Available:  1


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices()

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
# Initialise generator with rescale factor 1./255
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=10,  zoom_range=0.1, horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255, rotation_range=10, zoom_range=0.1, horizontal_flip=True, validation_split=0.5)

# Preprocess training set
train_generator = train_gen.flow_from_directory(
    fer_directory + '/train',
    target_size=(48,48),
    batch_size=128,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)
# Preprocess test set
test_generator = test_gen.flow_from_directory(
    fer_directory + '/test',
    target_size=(48,48),
    batch_size=128,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training',
    shuffle=False
)

validation_generator = test_gen.flow_from_directory(
    fer_directory + '/test',
    target_size=(48,48),
    batch_size=128,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation',
    shuffle=True
)


Found 28709 images belonging to 7 classes.
Found 3591 images belonging to 7 classes.
Found 3587 images belonging to 7 classes.


<h2>Model Selection</h2>

In [ ]:
def create_model(hp):
    # Define model
    model = Sequential()

    hp_conv_filter_1 = hp.Int('conv_filter_1', min_value=64, max_value=512, step=64)
    hp_conv_filter_2 = hp.Int('conv_filter_2', min_value=64, max_value=512, step=64)
    hp_conv_filter_3 = hp.Int('conv_filter_3', min_value=64, max_value=512, step=64)
    hp_conv_dropout = hp.Float('conv_dropout', min_value=0.0, max_value=0.5, step=0.1)
    # 1st convolution layer
    model.add(Conv2D(hp_conv_filter_1, (3,3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # 2nd convolution layer
    model.add(Conv2D(hp_conv_filter_2, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # 3rd convolution layer
    model.add(Conv2D(hp_conv_filter_3, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # Flatten and feed into dense layer
    model.add(Flatten())
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    # Tune dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(Dense(hp_units))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hp_dropout))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [4]:
def create_model(hp):
    # Define model
    model = Sequential()

    hp_conv_filter_1 = hp.Int('conv_filter_1', min_value=64, max_value=512, step=64)
    hp_conv_filter_2 = hp.Int('conv_filter_2', min_value=64, max_value=512, step=64)
    hp_conv_filter_3 = hp.Int('conv_filter_3', min_value=64, max_value=512, step=64)
    hp_conv_dropout = hp.Float('conv_dropout', min_value=0.0, max_value=0.5, step=0.1)
    # 1st convolution layer
    model.add(Conv2D(hp_conv_filter_1, (3,3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # 2nd convolution layer
    model.add(Conv2D(hp_conv_filter_2, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # 3rd convolution layer
    model.add(Conv2D(hp_conv_filter_3, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(hp_conv_dropout))

    # Flatten and feed into dense layer
    model.add(Flatten())
    # Tune the number of units in the dense layer
    hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
    # Tune dropout rate
    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    model.add(Dense(hp_units))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(hp_dropout))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    # Tune learning rate
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    model.compile(optimizer=Adam(learning_rate=hp_learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='./first_model_experiment',
    project_name='facial_expression_recognition'
)

# Stop training when validation loss stops improving
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tuner.search(train_generator, epochs=20, validation_data=validation_generator, callbacks=[stop_early])

best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

# Print results
print(best_hps)

Trial 30 Complete [00h 45m 18s]
val_accuracy: 0.5331823825836182

Best val_accuracy So Far: 0.5742902159690857
Total elapsed time: 10h 09m 32s


In [5]:
print(best_hps.values)

{'conv_filter_1': 64, 'conv_filter_2': 320, 'conv_filter_3': 512, 'conv_dropout': 0.0, 'units': 512, 'dropout': 0.30000000000000004, 'learning_rate': 0.0001, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}


In [5]:
def create_ref_model(hp):
    # Reference model
    ref_model = Sequential()

    hp_dropout = hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)
    hp_conv_filters_1 = hp.Int('conv_filters_1', min_value=64, max_value=512, step=64)
    hp_conv_filters_2 = hp.Int('conv_filters_2', min_value=64, max_value=512, step=64)
    hp_conv_filters_3 = hp.Int('conv_filters_3', min_value=64, max_value=512, step=64)
    hp_conv_filters_4 = hp.Int('conv_filters_4', min_value=64, max_value=512, step=64)
    hp_conv_filters_5 = hp.Int('conv_filters_5', min_value=64, max_value=512, step=64)
    ref_model.add(Conv2D(hp_conv_filters_1, (3,3), padding='same', input_shape=(48,48,1), activation='relu'))
    ref_model.add(Conv2D(hp_conv_filters_2, (3,3), padding='same', activation='relu'))
    ref_model.add(BatchNormalization())
    ref_model.add(MaxPooling2D(pool_size=(2,2)))
    ref_model.add(Dropout(hp_dropout))

    ref_model.add(Conv2D(hp_conv_filters_3, (3,3), padding='same', activation='relu'))
    ref_model.add(BatchNormalization())
    ref_model.add(MaxPooling2D(pool_size=(2,2)))
    ref_model.add(Dropout(hp_dropout))

    ref_model.add(Conv2D(hp_conv_filters_4, (3,3), padding='same', activation='relu'))
    ref_model.add(BatchNormalization())
    ref_model.add(MaxPooling2D(pool_size=(2,2)))
    ref_model.add(Dropout(hp_dropout))

    ref_model.add(Conv2D(hp_conv_filters_5, (3,3), padding='same', activation='relu'))
    ref_model.add(BatchNormalization())
    ref_model.add(MaxPooling2D(pool_size=(2,2)))
    ref_model.add(Dropout(hp_dropout))

    hp_dense_filter = hp.Int('dense_filter', min_value=32, max_value=512, step=32)
    ref_model.add(Flatten())
    ref_model.add(Dense(hp_dense_filter, activation='relu'))
    ref_model.add(BatchNormalization())

    hp_dense_dropout = hp.Float('dense_dropout', min_value=0.0, max_value=0.5, step=0.1)
    ref_model.add(Dropout(hp_dense_dropout))
    ref_model.add(Dense(7, activation='softmax'))

    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    ref_model.compile(optimizer=Adam(learning_rate=hp_learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return ref_model

In [9]:
ref_tuner = kt.Hyperband(
    create_ref_model,
    objective='val_accuracy',
    max_epochs=20,
    factor=3,
    directory='./ref_model_experiment',
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

ref_tuner.search(train_generator, epochs=20, validation_data=validation_generator, callbacks=[stop_early])

print (ref_tuner.get_best_hyperparameters(num_trials=1)[0].values)

Trial 30 Complete [00h 40m 48s]
val_accuracy: 0.5725483298301697

Best val_accuracy So Far: 0.6093015074729919
Total elapsed time: 05h 22m 46s
{'dropout': 0.30000000000000004, 'conv_filters_1': 256, 'conv_filters_2': 384, 'conv_filters_3': 320, 'conv_filters_4': 192, 'conv_filters_5': 192, 'dense_filter': 352, 'dense_dropout': 0.1, 'learning_rate': 0.001, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0019'}


<h2>Model Selection</h2>

In [4]:
def create_model(hp):
    model = Sequential()
    # 1st convolution layer
    hp_conv_filter_1 = hp.Int('filter layer 1', min_value=32, max_value=512, step=32)
    model.add(Conv2D(hp_conv_filter_1, (3,3), padding='same', input_shape=(48,48,1), activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("1st layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("1st layer with " + str(i) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # 2nd Convolution Layer
    model.add(Conv2D(hp.Int("filter layer 2", min_value=32, max_value=512, step=32), (5,5), padding='same', activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("2nd layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("2nd layer with " + str(i) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # 3rd Convolution Layer
    model.add(Conv2D(hp.Int("filter layer 3", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("3rd layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("3rd layer with " + str(i) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # 4th Convolution Layer
    model.add(Conv2D(hp.Int("filter layer 4", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("4th layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("4th layer with " + str(i) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # 5th Convolution Layer
    model.add(Conv2D(hp.Int("filter layer 5", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("5th layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("5th layer with " + str(i) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # Flatten and feed into dense layer
    model.add(Flatten())
    # Tune number of dense layers from 1 to 3
    for i in range(1, hp.Int("dense layer", 1, 3)):
        model.add(Dense(hp.Int("dense layer " + str(i), min_value=128, max_value=1024, step=64), activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
model_tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='./model_selection_experiment_2',
    overwrite=False, # Reload from checkpoint
    project_name="fer"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model_tuner.search(train_generator, epochs=50, validation_data=validation_generator, callbacks=[stop_early])

Trial 2 Complete [00h 11m 15s]

Best val_accuracy So Far: None
Total elapsed time: 00h 11m 43s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
448               |480               |filter layer 1
1                 |2                 |1st layer additional
320               |32                |filter layer 2
0                 |2                 |2nd layer additional
384               |416               |filter layer 3
0                 |1                 |3rd layer additional
480               |32                |filter layer 4
1                 |0                 |4th layer additional
480               |192               |filter layer 5
2                 |0                 |5th layer additional
2                 |2                 |dense layer
256               |32                |1st layer with 0 extra layer
64                |32                |1st layer with 1 extra layer
256               |32                |2nd layer with 0 extra layer
160           

Traceback (most recent call last):
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\tuner.py", lin

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 273, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 238, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\tuners\hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\Python\Python38\lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "c:\Users\subse\AppData\Local\Programs\P

In [6]:
print(model_tuner.get_best_hyperparameters(num_trials=1)[0].values)

{'filter layer 1': 96, '1st layer additional': 1, 'filter layer 2': 224, '2nd layer additional': 0, 'filter layer 3': 32, '3rd layer additional': 1, 'filter layer 4': 416, '4th layer additional': 0, 'filter layer 5': 480, '5th layer additional': 0, 'dense layer': 1, '1st layer with 0 extra layer': 480, '1st layer with 1 extra layer': 320, '5th layer with 0 extra layer': 160, 'dense layer 1': 640, 'dense layer 2': 320, '2nd layer with 0 extra layer': 480, '2nd layer with 1 extra layer': 448, '3rd layer with 0 extra layer': 480, '3rd layer with 1 extra layer': 320, '4th layer with 0 extra layer': 320, '5th layer with 1 extra layer': 224, '4th layer with 1 extra layer': 128, 'tuner/epochs': 15, 'tuner/initial_epoch': 5, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0020'}


<h2>Tune Hyperparameters of model selection</h2>

In [5]:
def create_model(hp):
    model = Sequential()

    hp_conv_dropout = hp.Float('conv_dropout', min_value=0.3, max_value=0.5, step=0.1)
    # 1st layer
    hp_kernel_1 = hp.Choice("conv_kernel_1", values=[3, 5])
    hp_kernel_2 = hp.Choice("conv_kernel_2", values=[3, 5])
    model.add(Conv2D(96, (hp_kernel_1, hp_kernel_1), padding='same', input_shape=(48,48,1), activation='relu', strides=hp.Choice('strides_1', values=[1,2])))
    model.add(Conv2D(480, (hp_kernel_2, hp_kernel_2), padding='same', activation='relu', strides=hp.Choice('strides_2', values=[1,2])))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(hp_conv_dropout))

    # 2nd layer
    hp_kernel_3 = hp.Choice("conv_kernel_3", values=[3, 5])
    model.add(Conv2D(224, (hp_kernel_3, hp_kernel_3), padding='same', activation='relu', strides=hp.Choice('strides_3', values=[1,2])))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(hp_conv_dropout))

    # 3rd layer
    hp_kernel_4 = hp.Choice("conv_kernel_4", values=[3, 5])
    hp_kernel_5 = hp.Choice("conv_kernel_5", values=[3, 5])
    model.add(Conv2D(32, (hp_kernel_4, hp_kernel_4), padding='same', activation='relu', strides=hp.Choice('strides_4', values=[1,2])))
    model.add(Conv2D(480, (hp_kernel_5, hp_kernel_5), padding='same', activation='relu', strides=hp.Choice('strides_5', values=[1,2])))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(hp_conv_dropout))

    #4th layer
    hp_kernel_6 = hp.Choice("conv_kernel_6", values=[3, 5])
    model.add(Conv2D(416, (hp_kernel_6, hp_kernel_6), padding='same', activation='relu', strides=hp.Choice('strides_6', values=[1,2])))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(hp_conv_dropout))

    # 5th layer
    hp_kernel_7 = hp.Choice("conv_kernel_7", values=[3, 5])
    model.add(Conv2D(480, (hp_kernel_7, hp_kernel_7), padding='same', activation='relu', strides=hp.Choice('strides_7', values=[1,2])))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2), padding='same'))
    model.add(Dropout(hp_conv_dropout))

    # Flatten and feed into dense layer
    model.add(Flatten())
    model.add(Dense(640, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(hp.Float('dense dropout', min_value=0.3, max_value=0.5, step=0.1)))


    model.add(Dense(7, activation='softmax'))
    optimizer_Choice = hp.Choice('optimizer', values=['adam', 'sgd', 'rmsprop', 'adagrad'])
    learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    if optimizer_Choice == 'adam':
        model.compile(optimizer=Adam(learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    elif optimizer_Choice == 'sgd':
        model.compile(optimizer=SGD(learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    elif optimizer_Choice == 'rmsprop':
        model.compile(optimizer=RMSprop(learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    elif optimizer_Choice == 'adagrad':
        model.compile(optimizer=Adagrad(learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
        
    return model

In [6]:
# Hyperband tuner
model_tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=40,
    factor=3,
    directory='./model_selection_experiment_3',
    overwrite=False, # Reload from checkpoint
    project_name="fer"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

model_tuner.search(train_generator, epochs=40, validation_data=validation_generator, callbacks=[stop_early, keras.callbacks.TensorBoard(log_dir='./logs')])

Trial 90 Complete [00h 19m 57s]
val_accuracy: 0.5597992539405823

Best val_accuracy So Far: 0.6339559555053711
Total elapsed time: 1d 12h 04m 33s


In [7]:
print(model_tuner.get_best_hyperparameters(num_trials=1)[0].values)

{'conv_dropout': 0.4, 'conv_kernel_1': 5, 'conv_kernel_2': 5, 'strides_1': 1, 'strides_2': 2, 'conv_kernel_3': 3, 'strides_3': 1, 'conv_kernel_4': 3, 'conv_kernel_5': 5, 'strides_4': 2, 'strides_5': 1, 'conv_kernel_6': 3, 'strides_6': 1, 'conv_kernel_7': 5, 'strides_7': 2, 'dense dropout': 0.3, 'optimizer': 'rmsprop', 'learning_rate': 0.001, 'tuner/epochs': 40, 'tuner/initial_epoch': 14, 'tuner/bracket': 3, 'tuner/round': 3, 'tuner/trial_id': '0047'}


<h2>Test hyperband with number of layers</h2>

In [4]:
def create_model(hp):
    model = Sequential()
    # 1st convolution layer
    hp_conv_filter_1 = hp.Int('filter layer 1', min_value=32, max_value=512, step=32)
    model.add(Conv2D(hp_conv_filter_1, (3,3), padding='same', input_shape=(48,48,1), activation='relu'))
    # Loop to see whether we need to add 0 to 2 more conv layers
    for i in range(0, hp.Int("1st layer additional", 0, 2)):
        model.add(Conv2D(hp.Int("1st layer with " + str(i + 1) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # Try adding 1 to 5 more conv layers
    for i in range(1, hp.Int("Additional hidden layers", 1, 5)):
        model.add(Conv2D(hp.Int("filter layer " + str(i+1), min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
        # Loop to see whether we need to add 0 to 2 more conv layers
        for j in range(0, hp.Int(str(i+1) + "th layer additional", 0, 2)):
            model.add(Conv2D(hp.Int(str(i+1) + "th layer with " + str(j + 1) + " extra layer", min_value=32, max_value=512, step=32), (3,3), padding='same', activation='relu'))
        model.add(BatchNormalization())
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.4))
    
    # Flatten and feed into dense layer
    model.add(Flatten())
     # Tune number of dense layers from 1 to 3
    for i in range(1, hp.Int("dense layer", 1, 3)):
        model.add(Dense(hp.Int("dense layer " + str(i+1), min_value=128, max_value=1024, step=64), activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.4))
    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    model.compile(optimizer=Adam(learning_rate=0.0001, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [8]:
model_tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='./model_selection_experiment_2',
    overwrite=False, # Reload from checkpoint
    project_name="fer"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model_tuner.search(train_generator, epochs=40, validation_data=validation_generator, callbacks=[stop_early])

Trial 70 Complete [00h 25m 14s]
val_accuracy: 0.6467800140380859

Best val_accuracy So Far: 0.676609992980957
Total elapsed time: 14h 45m 21s

Search: Running Trial #71

Value             |Best Value So Far |Hyperparameter
224               |128               |filter layer 1
1                 |2                 |1st layer additional
3                 |3                 |Additional hidden layers
2                 |2                 |dense layer
256               |512               |filter layer 2
1                 |2                 |2th layer additional
96                |512               |1st layer with 1 extra layer
32                |352               |2th layer with 1 extra layer
288               |288               |2th layer with 2 extra layer
96                |384               |filter layer 3
1                 |0                 |3th layer additional
288               |64                |filter layer 4
0                 |0                 |4th layer additional
352            

KeyboardInterrupt: 

In [2]:
model_tuner = kt.Hyperband(
    create_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='./model_selection_experiment_2',
    overwrite=False, # Reload from checkpoint
    project_name="fer"
)

stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model_tuner.search(train_generator, epochs=40, validation_data=validation_generator, callbacks=[stop_early])

NameError: name 'kt' is not defined

In [5]:
# Get the optimal hyperparameters
best_hps=model_tuner.get_best_hyperparameters(num_trials=1)[0]
best_hps.values

NameError: name 'model_tuner' is not defined

In [9]:
model = model_tuner.hypermodel.build(best_hps)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_11 (Conv2D)          (None, 48, 48, 128)       1280      
                                                                 
 conv2d_12 (Conv2D)          (None, 48, 48, 512)       590336    
                                                                 
 conv2d_13 (Conv2D)          (None, 48, 48, 512)       2359808   
                                                                 
 batch_normalization_7 (Batc  (None, 48, 48, 512)      2048      
 hNormalization)                                                 
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 24, 24, 512)      0         
 2D)                                                             
                                                                 
 dropout_7 (Dropout)         (None, 24, 24, 512)      